# A comparison of local search approaches to the 8-queen problem
---
Here we will analyze, a solution to 8-queen problem by greedy descent, greedy descent with the help of sideway moves and finally Simulated annealing and compare their completeness.


## Problem Description
Given a standard 8x8 chessboard, Place 8 queens on a chess board so that no queen attacks another.. So essentially,
* no adjacent pieces in the same row
* pieces in the same diagonals
* no adjacent pieces in the same column


In [1]:
#Importing libraries
import math, random

## Class Documentation: Board
#### Description

The Board class represents a chessboard with queens placed on it. It provides functionality to initialize the board, create duplicates, and calculate the number of attacks between queens.

#### Constructor
`__init__(self, size: int)`

- **Parameters**: size (int): The size of the chessboard, indicating the number of rows and columns.
- **Description**: Initializes a new instance of the Board class with the specified size. Places queens randomly on the board during initialization.

#### Methods
`duplicate(self) -> Board`

- **Returns**: A new instance of the Board class, which is an identical copy of the current board.
- **Description**: Creates a duplicate board with the same size and queen placements.

`number_of_attacks(self) -> int`

- **Returns**: The total number of queen attacks on the current board.
- **Description**: Calculates and returns the number of attacks between queens on the board. Attacks include queens in the same row, column, or diagonals.

In [2]:
class Board:
    def __init__(self, size):
        self.size = size
        self.queens = []
        for i in range(self.size):
            self.queens.append(random.randint(0, self.size - 1))

    def duplicate(self):
        duplicate = Board(self.size)
        duplicate.queens = self.queens.copy()
        return duplicate

    def number_of_attacks(self) -> int:
        attacks = 0
        for queen in range(self.size-1):
            for next_queen in range(queen + 1,self.size):
                if(self.queens[queen] == self.queens[next_queen]): attacks += 1
                queen_diff = abs(queen - next_queen)
                if(self.queens[next_queen] == self.queens[queen] - queen_diff or self.queens[next_queen] == self.queens[queen] + queen_diff): attacks += 1
        return attacks

## Greedy Descent(aka Hill climbing Search)
Number of attacks is used as the heuristic to determine the best neighbour. If the algorithm can descends to 0 attacks, it is considered a success whereas if the neighbour is more inefficient than current positions, then the search is abandoned as False. Greedy descent search suffers from shoulders and plateaus which cause it to be stuck in local optimas

Approach inspired from my professor Ajay Anand's class


In [3]:
def best_neighbour(board: Board, value: int):
    bestVal = value
    bestPos = [0, board.queens[0]]
    restore_to_original = board.queens #this is for purely readability purposes and is not efficient
    newBoard = board.duplicate()
    for columns in range(newBoard.size):
        for rows in range(newBoard.size):
            newBoard.queens[columns] = rows
            newVal = newBoard.number_of_attacks()
            if newVal < bestVal:
                bestVal = newVal
                bestPos[0] = columns; bestPos[1] = rows
            newBoard.queens = list(restore_to_original)
    newBoard.queens[bestPos[0]] = bestPos[1]
    return newBoard

def greedy_descent(board: Board):
    current = board.duplicate()
    currentVal = current.number_of_attacks()
    while True:
        neighbour = best_neighbour(current, currentVal)
        neighbourVal = neighbour.number_of_attacks()
        if neighbourVal == 0: return True
        if neighbourVal >= currentVal: return False
        current = neighbour
        currentVal = neighbourVal

### Greedy Descent with Sideway Moves
The problem of shoulders can be partially alleviated by introducing sideway moves which allow the algorithm to continue over moves of similar efficiency as the current position for a limited number of moves in hopes of finding a more efficient solution

In [4]:
def best_neighbour_sm(board: Board, value: int, sideways_limit: int):
    bestVal = value
    bestPos = [0, board.queens[0]]
    restore_to_original = board.queens #this is for purely readability purposes and is not efficient
    newBoard = board.duplicate()
    sideways_moves = 0
    for columns in range(newBoard.size):
        for rows in range(newBoard.size):
            newBoard.queens[columns] = rows
            newVal = newBoard.number_of_attacks()
            if newVal < bestVal:
                bestVal = newVal
                bestPos[0] = columns; bestPos[1] = rows
                sideways_moves = 0
            elif newVal == bestVal and sideways_moves < sideways_limit:
                sideways_moves += 1
                if random.random() < 0.5:
                    bestPos[0] = columns; bestPos[1] = rows
            newBoard.queens = list(restore_to_original)
    newBoard.queens[bestPos[0]] = bestPos[1]
    return newBoard


def greedy_descent_using_sideway_moves(board: Board):
    current = board.duplicate()
    currentVal = current.number_of_attacks()
    while True:
        neighbour = best_neighbour_sm(current, currentVal, 10)
        neighbourVal = neighbour.number_of_attacks()
        if neighbourVal == 0: return True
        if neighbourVal >= currentVal: return False
        current = neighbour
        currentVal = neighbourVal


## Simulated Annealing
An innovative and efficient solution by Kirkpatrick et al. It follows the analogy of metallurgical annealing, allowing for greater randomness in higher temperatures while slowly cooling down to an optima in lower temperatures. This allows the algorithm to take to uphill values which otherwise would have been rejected.
Solution inspired from Ajay Anand and Russel-Norvig book

In [5]:
def simulated_annealing(board: Board):
    current = board.duplicate()
    current_attacked = current.number_of_attacks()
    temperature = 1.0
    while temperature > 0:
        new_board = random_neighbour(current)
        new_board_attacked = new_board.number_of_attacks()
        energy_diff = new_board_attacked - current_attacked
        if energy_diff < 0 or random.random() < math.exp(-energy_diff/ temperature):
            current = new_board
            current_attacked = new_board_attacked
            #print(f"{current.queens} at {current_attacked} attacks in {temperature}\n")
            if current_attacked == 0: return True
        temperature = round(temperature*0.99, 3)
    return False


def random_neighbour(board: Board):
    new_board = board.duplicate()
    i = random.randint(0, board.size-1)
    j = random.randint(0, board.size-1)
    new_board.queens[i] = j
    return new_board

# Tests

The algorithms are run a number of times and the percentage by which they are successful is recorded. Due to compute restrictions, I couldn't scale it up much

In [6]:
def main():
    total_attempts = 10
    sa = 0
    gd = 0
    gdsm = 0

    for _ in range(total_attempts):
        board = Board(8)
        if simulated_annealing(board):
            sa += 1

        if greedy_descent(board):
            gd += 1

        if greedy_descent_using_sideway_moves(board):
          gdsm += 1

    sa_percent = (sa / total_attempts) * 100
    gd_percent = (gd / total_attempts) * 100
    gdsm_percent = (gdsm / total_attempts) * 100

    print(f"Simulated Annealing Success Percentage: {sa_percent}%")
    print(f"Greedy Descent Success Percentage: {gd_percent}%")
    print(f"Greedy Descent w/ Sideway Moves Success Percentage: {gdsm_percent}%")

if __name__ == "__main__":
    main()

Simulated Annealing Success Percentage: 100.0%
Greedy Descent Success Percentage: 0.0%
Greedy Descent w/ Sideway Moves Success Percentage: 20.0%


# Conclusion
Simulated annealing performs excellently and is complete. Greedy descent is incomplete, while Sideway Moves do introduce some marginal improvements, it is incomplete as well.